In [ ]:
ptbr 0.626~0.71~0.740
ptpt 0.706~0.73~0.784

Outras similaridades: Cos, Ovl, Jacc, Dice

Semantic Similarity

These models find semantically similar sentences within one language or across languages:

**distiluse-base-multilingual-cased-v2**: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. While the original mUSE model only supports 16 languages, this multilingual knowledge distilled version supports 50+ languages.

**xlm-r-distilroberta-base-paraphrase-v1** - Multilingual version of distilroberta-base-paraphrase-v1, trained on parallel data for 50+ languages.

**xlm-r-bert-base-nli-stsb-mean-tokens**: Produces similar embeddings as the bert-base-nli-stsb-mean-token model. Trained on parallel data for 50+ languages.

**distilbert-multilingual-nli-stsb-quora-ranking** - Multilingual version of distilbert-base-nli-stsb-quora-ranking. Fine-tuned with parallel data for 50+ languages.

**T-Systems-onsite/cross-en-de-roberta-sentence-transformer** - Multilingual model for English an German. [More]

# Imports e métodos necessários

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import pandas as pd 
import xml.etree.ElementTree as et 

from scipy.stats import pearsonr

def parse_xml(xml_file):
    """Parse xml to pandas dataframe."""
    xtree = et.parse(xml_file)
    xroot = xtree.getroot() 

    df_cols = ['id', 't', 'h', 'similarity']
    rows = []

    for node in xroot:
        id_ = node.attrib.get("id")
        similarity = node.attrib.get("similarity")
        t = node.find("t").text
        h = node.find("h").text

        rows.append({
            "id": id_,
            "t": t, 
            "h": h,
            "similarity": similarity
        })
    return pd.DataFrame(rows, columns=df_cols, dtype=float)

def eval_similarity(pairs_gold, pairs_sys):
    '''
    Evaluate the semantic similarity output of the system against a gold score. 
    Results are printed to stdout.
    '''
    
    gold_values = np.array(pairs_gold)
    sys_values = np.array(pairs_sys)
    pearson = pearsonr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()
    
    print()
    print('Similarity evaluation')
    print('Pearson\t\tMean Squared Error')
    print('-------\t\t------------------')
    print('{:7.3f}\t\t{:18.2f}'.format(pearson, mse))

# Carregando os dados

In [3]:
!ls ../data/assin

assin-ptbr-dev.xml   assin-ptbr-train.xml  assin-ptpt-test.xml
assin-ptbr-test.xml  assin-ptpt-dev.xml    assin-ptpt-train.xml


In [4]:
df_ptbr_train = parse_xml('../data/assin/assin-ptbr-train.xml')
df_ptbr_dev = parse_xml('../data/assin/assin-ptbr-dev.xml')
df_ptbr_test = parse_xml('../data/assin/assin-ptbr-test.xml')

df_ptpt_train = parse_xml('../data/assin/assin-ptpt-train.xml')
df_ptpt_dev = parse_xml('../data/assin/assin-ptpt-dev.xml')
df_ptpt_test = parse_xml('../data/assin/assin-ptpt-test.xml')

In [5]:
print(f'assin-ptbr-train: {df_ptbr_train.shape}')
print(f'assin-ptbr-dev: {df_ptbr_dev.shape}')
print(f'assin-ptbr-test: {df_ptbr_test.shape}')
print()
print(f'assin-ptpt-train: {df_ptpt_train.shape}')
print(f'assin-ptpt-dev: {df_ptpt_dev.shape}')
print(f'assin-ptpt-test: {df_ptpt_test.shape}')

assin-ptbr-train: (2500, 4)
assin-ptbr-dev: (500, 4)
assin-ptbr-test: (2000, 4)

assin-ptpt-train: (2500, 4)
assin-ptpt-dev: (500, 4)
assin-ptpt-test: (2000, 4)


In [6]:
df_ptbr_train.head()

,id,t,h,similarity
0,1.0,"A gente faz o aporte financeiro, é como se a e...",Fernando Moraes afirma que não tem vínculo com...,2.00
1,2.0,"Em 2013, a história de como Walt Disney conven...",P.L.Travers era completamente contra a adaptaç...,2.25
2,3.0,"David Silva bateu escanteio, Kompany escalou a...","David Silva cobrou escanteio, o zagueiro se ap...",3.75
3,4.0,"Para os ambientalistas, as metas anunciadas pe...","Dilma aproveitou seu discurso ontem, na Confer...",2.75
4,5.0,"De acordo com a PM, por volta das 10h30 havia ...",O protesto encerrou por volta de 12h15 (horári...,2.00


# Testes pt-br

## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### distiluse-base-multilingual-cased-v2

In [7]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)

for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_finetuning_assin_distiluse-base-multilingual-cased-v2-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('distiluse-base-multilingual-cased-v2')


    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.789		              0.29
Spearman correlation:   0.770





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.789		              0.30
Spearman correlation:   0.771





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.789		              0.30
Spearman correlation:   0.770





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.788		              0.30
Spearman correlation:   0.768





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.788		              0.30
Spearman correlation:   0.769





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.785		              0.30
Spearman correlation:   0.767





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.786		              0.30
Spearman correlation:   0.767





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.789		              0.30
Spearman correlation:   0.770





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.787		              0.30
Spearman correlation:   0.769





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.787		              0.30
Spearman correlation:   0.768


### xlm-r-distilroberta-base-paraphrase-v1

In [8]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)

for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_finetuning_assin_xlm-r-distilroberta-base-paraphrase-v1-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')


    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30
Spearman correlation:   0.781





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.30
Spearman correlation:   0.781





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30
Spearman correlation:   0.781





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.789		              0.30
Spearman correlation:   0.777





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.791		              0.30
Spearman correlation:   0.780





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.794		              0.30
Spearman correlation:   0.782





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.791		              0.30
Spearman correlation:   0.780





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.30
Spearman correlation:   0.781





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30
Spearman correlation:   0.780





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.30
Spearman correlation:   0.782


### xlm-r-bert-base-nli-stsb-mean-tokens

In [ ]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        
for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_finetuning_assin_xlm-r-bert-base-nli-stsb-mean-tokens-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')




    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.749		              0.38
Spearman correlation:   0.733





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.750		              0.38
Spearman correlation:   0.733





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.746		              0.39
Spearman correlation:   0.730





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.748		              0.38
Spearman correlation:   0.732





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.746		              0.39
Spearman correlation:   0.730


In [7]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        
for i in range(6, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_finetuning_assin_xlm-r-bert-base-nli-stsb-mean-tokens-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')




    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.747		              0.39
Spearman correlation:   0.731





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.747		              0.39
Spearman correlation:   0.731





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.749		              0.38
Spearman correlation:   0.733





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.748		              0.39
Spearman correlation:   0.731





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.748		              0.39
Spearman correlation:   0.732


### distilbert-multilingual-nli-stsb-quora-ranking

In [8]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        
for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_finetuning_assin_xlm-r-bert-base-nli-stsb-mean-tokens-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')




    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        

for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_finetuning_assin_distilbert-multilingual-nli-stsb-quora-ranking-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.734		              0.39
Spearman correlation:   0.724





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.731		              0.39
Spearman correlation:   0.720





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.39
Spearman correlation:   0.721





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.732		              0.39
Spearman correlation:   0.722





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.731		              0.40
Spearman correlation:   0.722





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.39
Spearman correlation:   0.722





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.731		              0.40
Spearman correlation:   0.720





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.39
Spearman correlation:   0.723





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.735		              0.39
Spearman correlation:   0.725





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.734		              0.39
Spearman correlation:   0.725


### T-Systems-onsite/cross-en-de-roberta-sentence-transformer

In [9]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptbr_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)


        
for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_finetuning_assin_T-Systems-onsite_cross-en-de-roberta-sentence-transformer-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Exception when trying to download http://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404
SentenceTransformer-Model http://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip not found. Try to create it from scratch
Try to create Transformer Model T-Systems-onsite/cross-en-de-roberta-sentence-transformer with mean pooling





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.784		              0.32
Spearman correlation:   0.774





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.786		              0.31
Spearman correlation:   0.775





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.783		              0.32
Spearman correlation:   0.773





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.785		              0.31
Spearman correlation:   0.775


...




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.785		              0.31
Spearman correlation:   0.775





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.785		              0.31
Spearman correlation:   0.774





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.783		              0.32
Spearman correlation:   0.773





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.786		              0.31
Spearman correlation:   0.776





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.786		              0.31
Spearman correlation:   0.776


# Testes pt-pt

## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### distiluse-base-multilingual-cased-v2

In [10]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptpt_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptpt_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptpt_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        
for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_ptpt_finetuning_assin_distiluse-base-multilingual-cased-v2-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)



    model = SentenceTransformer('distiluse-base-multilingual-cased-v2')


    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.777		              0.31
Spearman correlation:   0.754





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.775		              0.31
Spearman correlation:   0.754





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.775		              0.31
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.774		              0.31
Spearman correlation:   0.752





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.774		              0.31
Spearman correlation:   0.752





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.777		              0.31
Spearman correlation:   0.756





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.774		              0.31
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.774		              0.31
Spearman correlation:   0.752





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.773		              0.32
Spearman correlation:   0.752





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.773		              0.32
Spearman correlation:   0.753


### xlm-r-distilroberta-base-paraphrase-v1

In [11]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptpt_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptpt_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptpt_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        
for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_ptpt_finetuning_assin_xlm-r-distilroberta-base-paraphrase-v1-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)



    model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.765		              0.34
Spearman correlation:   0.750





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.770		              0.33
Spearman correlation:   0.756





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.768		              0.33
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.769		              0.33
Spearman correlation:   0.754





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.768		              0.33
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.768		              0.33
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.767		              0.33
Spearman correlation:   0.751





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.769		              0.33
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.770		              0.33
Spearman correlation:   0.755





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.770		              0.33
Spearman correlation:   0.755


### xlm-r-bert-base-nli-stsb-mean-tokens

In [12]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptpt_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptpt_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptpt_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        
for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_ptpt_finetuning_assin_xlm-r-bert-base-nli-stsb-mean-tokens-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)

    model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.42
Spearman correlation:   0.717





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.42
Spearman correlation:   0.717





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.736		              0.41
Spearman correlation:   0.719





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.735		              0.41
Spearman correlation:   0.718





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.734		              0.42
Spearman correlation:   0.718





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.736		              0.41
Spearman correlation:   0.720





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.735		              0.41
Spearman correlation:   0.719





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.734		              0.41
Spearman correlation:   0.717





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.733		              0.42
Spearman correlation:   0.716





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.736		              0.41
Spearman correlation:   0.720


### distilbert-multilingual-nli-stsb-quora-ranking

In [13]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptpt_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptpt_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptpt_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)

for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_ptpt_finetuning_assin_distilbert-multilingual-nli-stsb-quora-ranking-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)

    model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.729		              0.39
Spearman correlation:   0.716





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.723		              0.40
Spearman correlation:   0.708





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.726		              0.39
Spearman correlation:   0.712





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.726		              0.39
Spearman correlation:   0.711





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.728		              0.39
Spearman correlation:   0.712





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.730		              0.39
Spearman correlation:   0.716





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.725		              0.39
Spearman correlation:   0.708





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.725		              0.39
Spearman correlation:   0.710





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.727		              0.39
Spearman correlation:   0.712





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.726		              0.39
Spearman correlation:   0.711


## T-Systems-onsite-cross-en-de-roberta

In [14]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
dev_samples = []
test_samples = []

for i, row in df_ptpt_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

for i, row in df_ptpt_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)

for i, row in df_ptpt_test.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    test_samples.append(inp_example)
        
for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'../data/sbert_ptpt_finetuning_assin_T-Systems-onsite_cross-en-de-roberta-sentence-transformer-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.770		              0.34
Spearman correlation:   0.756





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.770		              0.34
Spearman correlation:   0.756





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.768		              0.34
Spearman correlation:   0.754





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.769		              0.34
Spearman correlation:   0.755





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.768		              0.34
Spearman correlation:   0.753





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.768		              0.34
Spearman correlation:   0.754





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.768		              0.34
Spearman correlation:   0.754





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.771		              0.34
Spearman correlation:   0.756





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.772		              0.34
Spearman correlation:   0.757





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.769		              0.34
Spearman correlation:   0.755
